# Simulation of non-linear BSDE 

## Set of parameters

In [297]:
sigma_alpha = 5
eta_alpha = 1
sigma_z = 5
eta_z = 1
kappa = .1 #big enough
phi = 0.001
b = .01
l = 1
alpha_I = 0.001
psi = alpha_I + b/2
T = .9
N = 20
N_steps = 1_000
Batch_size = 100
kappa_z = 1
print(psi)

0.006


## Implementation

In [298]:
import torch
import torch.nn as nn

class ZNet(nn.Module):
    def __init__(self, input_dim, output_dim=1, hidden_dim=64):
        super(ZNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, q_t,alpha_t,z_t):
        xy = torch.cat([q_t,alpha_t,z_t], dim=1)
        return self.net(xy)


In [299]:
def simulate_trajectory(Q0_I,Q0_U, Y0,Z_nets, 
                        b_QI,b_QU,b_Y, T, N, dW_S,dW_A,dW_Z):
    dt = T / N
    Q_I,Q_U, Y_I,Z_vals_S,Z_vals_A,Z_vals_Z,alpha,Z= [Q0_I],[Q0_U], [Y0],[],[],[], [torch.zeros(1, 1).expand(Q0_I.shape[0], -1)],[torch.zeros(1, 1).expand(Q0_I.shape[0], -1)]

    for i in range(N):
        t = i * dt
        q_I,q_U, y_I,Alpha_t,Z_t = Q_I[-1],Q_U[-1], Y_I[-1],alpha[-1],Z[-1]
        
        z_S = Z_nets[0][i](q_I,Alpha_t,Z_t)
        z_A = Z_nets[1][i](q_I,Alpha_t,Z_t)
        z_Z = Z_nets[2][i](q_I,Alpha_t,Z_t)
        
        Z_vals_S.append(z_S)
        Z_vals_A.append(z_A)
        Z_vals_Z.append(z_Z)
        
        dWt_S = dW_S[:, i, :]
        dWt_A = dW_A[:, i, :]
        dWt_Z = dW_Z[:, i, :]
        
        q_I_next = q_I + b_QI(l,q_U,y_I) * dt 
        q_U_next = q_U + b_QU(Z_t) * dt
        
        y_I_next = y_I + b_Y(q_I,Alpha_t) * dt + z_S * dWt_S + z_A * dWt_A + z_Z*dWt_Z
        alpha_next = Alpha_t - eta_alpha * Alpha_t * dt + dWt_A*sigma_alpha
        z_next = Z_t - eta_z*Z_t*dt + dWt_Z*sigma_z
        
        alpha.append(alpha_next)
        Z.append(z_next)
        Q_I.append(q_I_next)
        Q_U.append(q_U_next)
        Y_I.append(y_I_next)    
        
    return Q_I,Q_U,Y_I,Z_vals_S,Z_vals_A,Z_vals_Z,alpha,Z



In [300]:
def compute_loss(Y_I_T,Q_I_T):
    target_I = -2*psi*Q_I_T
    return torch.mean((Y_I_T - target_I) ** 2)


In [301]:
def train(Z_nets, Y0_I, Q0_I, Q0_U, b_QI, b_QU, b_Y,T, N, num_steps=N_steps, batch_size=Batch_size):
    optimizer = torch.optim.Adam(
        [{'params': net.parameters()} for net_list in Z_nets for net in net_list] + [{'params': [Y0_I]}], 
        lr=1e-3
    )

    for step in range(num_steps):
        dW_S = torch.randn(batch_size, N, Y0_I.shape[1]) * (T / N) ** 0.5
        dW_A = torch.randn(batch_size, N, Y0_I.shape[1]) * (T / N) ** 0.5
        dW_Z = torch.randn(batch_size, N, Y0_I.shape[1]) * (T / N) ** 0.5


        Q_I_list,Q_U_list, Y_I_list,_,_,_,_,_ = simulate_trajectory(
            Q0_I.expand(batch_size, -1), Q0_U.expand(batch_size,-1), Y0_I.expand(batch_size, -1),
            Z_nets, b_QI,b_QU,b_Y, T, N, dW_S,dW_A,dW_Z
        )
        loss = compute_loss(Y_I_list[-1], Q_I_list[-1],)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.item():.5f}")

    return dW_S,dW_A,dW_Z
            

In [302]:
# Fonctions dynamiques
def b_QI(l,q_U,y_I):
    c = -l-b*q_U + y_I
    tilde_c = -l+b*q_U - y_I

    left = torch.where(
        c > 0,
       c/(2*kappa),
        torch.zeros_like(c)
    )
    right = torch.where(
        tilde_c > 0,
        tilde_c/(2*kappa),
        torch.zeros_like(tilde_c)
    )
    return left - right

def b_QU(Z_t):
    return kappa_z*Z_t

def b_Y(q_I, alpha):
    return 2 * phi * q_I - alpha


In [ ]:
# Initialisation
dim_q_I = 1
dim_q_U = 1
dim_y = 1
input_dim = dim_q_I + dim_q_U + dim_y


# Z_nets: 4 groupes de N réseaux chacun
Z_nets = [
    [ZNet(input_dim=input_dim) for _ in range(N)]  
    for _ in range(3)
]

Q0_I = torch.zeros(1, dim_q_I)
Q0_U = torch.zeros(1, dim_q_U)
#Y0_I = nn.Parameter(torch.ones(1, dim_y))
Y0_I = nn.Parameter(torch.zeros(1, dim_y))


dW_S, dW_A,dW_Z = train(Z_nets, Y0_I, Q0_I, Q0_U, b_QI, b_QU, b_Y,T, N)


Step 0, Loss: 2.34665
Step 100, Loss: 0.19266
Step 200, Loss: 0.05589


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fonction pour afficher les courbes après l'entraînement
def plot_trajectory(Q_I, Q_U, Y_I, Z, alpha, T, N, batch_idx=0,batch_size=Batch_size):
    # Création d'un vecteur de temps
    time_grid = [i * T / N for i in range(N+1)]

    # Récupération des valeurs pour un élément spécifique du batch
    Q_I_vals = [Q_I[t][batch_idx].item() for t in range(N+1)]
    Q_U_vals = [Q_U[t][batch_idx].item() for t in range(N+1)]
    Y_I_vals = [Y_I[t][batch_idx].item() for t in range(N+1)]
    alpha_vals = [alpha[t][batch_idx].item() for t in range(N+1)]
    Z_vals = [Z[t][batch_idx].item() for t in range(N+1)]
    
    
    alpha_values=[]
    for t in range(N+1):
        alpha_values.append(alpha_vals[t]*(1-np.exp(-eta_alpha*(T-time_grid[t])))/eta_alpha)
        
    true_nu_plus = []
    true_nu_minus=[]
    tilde_delta_plus = []
    tilde_delta_minus = []
    delta_plus = []
    delta_minus = []
    c=[]
    tilde_c =[]
    for t in range(N+1):
        delta_plus.append(l+b*Q_U_vals[t]+b*Q_I_vals[t])
        delta_minus.append(l-b*Q_U_vals[t]-b*Q_I_vals[t])

        tilde_delta_plus.append(l+b*Q_U_vals[t])
        tilde_delta_minus.append(l-b*Q_U_vals[t])

        c.append(-tilde_delta_plus[t]+Y_I_vals[t])
        tilde_c.append(-tilde_delta_minus[t]-Y_I_vals[t])
        

    nu_plus = []
    nu_minus = []
   
    for t in range(N+1):
        if -l+alpha_vals[t]*(1-np.exp(-eta_alpha*(T-time_grid[t])))/eta_alpha > 0:
            true_nu_plus.append((-l+alpha_vals[t]*(1-np.exp(-eta_alpha*(T-time_grid[t])))/eta_alpha)/(2*kappa))
        else:
            true_nu_plus.append(0)
            
    for t in range(N+1):
        if -l-alpha_vals[t]*(1-np.exp(-eta_alpha*(T-time_grid[t])))/eta_alpha > 0:
            true_nu_minus.append((-l-alpha_vals[t]*(1-np.exp(-eta_alpha*(T-time_grid[t])))/eta_alpha)/(2*kappa))
        else:
            true_nu_minus.append(0)


    for t in range(N+1):
        if c[t] <= 0:
            nu_plus.append(0)
        else:
            nu_plus.append(c[t]/(2*kappa))
    
    for t in range(N+1):
        if tilde_c[t] <= 0:
            nu_minus.append(0)
        else:
            nu_minus.append(tilde_c[t]/(2*kappa))
            
    
    # Tracer les courbes pour Q_I, Q_M, Y_I, Y_M
    plt.figure(figsize=(12, 6))

    # Trajectoire pour Q_I et Q_M
    plt.plot(time_grid, nu_plus, label="\nu_t^+", color='blue')
    plt.plot(time_grid, nu_minus, label="\nu_t^-", color='orange')
    plt.title("Trajectoire de trading speed")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.plot(time_grid,Z_vals,label='Z_t')
    plt.title("Trajectoire de trading speed uninf")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.plot(time_grid, true_nu_plus, label="\nu_t^+", color='blue')
    plt.plot(time_grid, true_nu_minus, label="\nu_t^-", color='orange')
    plt.title("Trajectoire du only alpha trading speed")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    
    plt.plot(time_grid,alpha_values,label='alpha_t')
    plt.plot(time_grid,Y_I_vals,label="NN_alpha")
    plt.title("Trajectoire de uninformed trading speed")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.plot(time_grid, Q_I_vals, label="Q_t^I", color='blue')
    plt.plot(time_grid, Q_U_vals, label="Q_t^U", color='orange')
    plt.title("Trajectoire de inventory")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.plot(time_grid, delta_plus, label="\delta_t^+", color='blue')
    plt.plot(time_grid, delta_minus, label="delta_t^-", color='orange')
    plt.title("Trajectoire quotes")
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()


# Fonction principale pour récupérer les valeurs et afficher les courbes
def plot_after_training(batch_size=Batch_size):
    # Récupérer la trajectoire simulée après l'entraînement
    Q_I, Q_U, Y_I, _, _,_, alpha,Z = simulate_trajectory(
        Q0_I.expand(batch_size, -1), Q0_U.expand(batch_size, -1), Y0_I.expand(batch_size, -1), Z_nets, b_QI, b_QU, b_Y, T, N,dW_S,dW_A,dW_Z)
    
    # Affichage des courbes pour le premier élément du batch

    plot_trajectory(Q_I, Q_U, Y_I, Z,alpha, T, N, batch_idx=0,batch_size=Batch_size)

# Appeler la fonction pour afficher les courbes après l'entraînement
plot_after_training()
